In [ ]:
import pandas as pd
import numpy as np
from astropy.units import Quantity
from astropy.cosmology import LambdaCDM, WMAP9
import matplotlib.pyplot as plt

import xga
# This just sets the number of cores this analysis is allowed to use
xga.NUM_CORES = 10
from xga.samples import ClusterSample
from xga.xspec import single_temp_apec
from xga.imagetools.misc import physical_rad_to_pix
from xga.exceptions import NoProductAvailableError
from xga import BLACKLIST
from xga.relations.clusters.MT import arnaud_m500, xxl_m500, xxl_cosmos_cccp_m500
from xga.relations.fit import scaling_relation_odr
from xga.models import power_law

In [ ]:
xcs_cosmo = LambdaCDM(70, 0.3, 0.7)
xxl_cosmo = WMAP9
locuss_cosmo = LambdaCDM(70, 0.3, 0.7)
lovisari_cosmo = LambdaCDM(70, 0.3, 0.7)

In [ ]:
locuss = pd.read_csv("sample_files/locuss_highlx_clusters.csv", dtype={'chandra_id': str, 'xmm_obsid': str})

In [ ]:
# Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
ra = locuss['ra'].values
dec = locuss['dec'].values
z = locuss['z'].values
n = locuss['name'].values
r500 = Quantity(locuss['r500'].values, 'kpc')
r2500 = Quantity(locuss['r2500'].values, 'kpc')


# Declaring the actual ClusterSample instance for the LoCuSS sample
locuss_srcs = ClusterSample(ra, dec, z, n, r500=r500, r2500=r2500, cosmology=locuss_cosmo, load_fits=True, 
                            use_peak=True, clean_obs=True, clean_obs_reg='r500', clean_obs_threshold=0.7)

In [ ]:
rel = locuss_srcs.mass_Tx('r500', Quantity(4, 'keV'), Quantity(5e+14, 'Msun'), )
rel.view(figsize=(6, 6))

In [ ]:
hyms = []
gms = []
txs = []
ezs = []
for src in locuss_srcs:
    try:
        mp = src.get_hydrostatic_mass_profiles()
        gm = mp.density_profile.gas_mass('simple_vikhlinin_dens', src.r500)[0]
        hym = mp.mass(src.r500)[0]
        
        hyms.append(hym)
        gms.append(gm)
        ezs.append(src.cosmo.efunc(src.redshift))
    except NoProductAvailableError:
        pass

hyms = Quantity(hyms)
gms = Quantity(gms)
ezs = np.array(ezs)

In [ ]:
y_arr = hyms.copy() #*ezs[..., None]
x_arr = gms.copy()
hym_gm_rel = scaling_relation_odr(power_law, y_arr[:, 0], y_arr[:, 1:], x_arr[:, 0], x_arr[:, 1:], 
                                  Quantity(5e+14, 'Msun'), Quantity(5e+13, 'Msun'))
hym_gm_rel.view(figsize=(6, 6))